In [328]:
import pandas as pd
import re
from unidecode import unidecode

from nltk.tokenize import word_tokenize
import spacy

from tqdm import tqdm

In [96]:
! python -m spacy download pt_core_news_sm

     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --------------------------------------- 0.0/13.0 MB 165.2 kB/s eta 0:01:19
     --------------------------------------- 0.0/13.0 MB 165.2 kB/s eta 0:01:19
     --------------------------------------- 0.0/13.0 MB 245.8 kB/s eta 0:00:53
     --------------------------------------- 0.1/13.0 MB 273.8 kB/s eta 0:00:48
     --------------------------------------- 0.1/13.0 MB 350.1 kB/s eta 0:00:37
     --------------------------------------- 0.1/13.0 MB 400.9 kB/s eta 0:00:33
      -------------------------------------- 0.2/13.0 MB 477.7 kB/s eta 0:00:27
      -------------------------------------- 0.2/13.0 MB 551.4 kB/s eta 0:00:24
      -------------------------------------- 0.2/13.0 MB 580.1 kB/s eta 0:00:22
      -------------------------------------- 0.3/13.0 MB 562.4 kB/s eta 0:00:23
      -------------------------------------- 0.3/13.0 MB

In [101]:
df = pd.read_parquet("datasets/joint/data.parquet.zstd")
df

,dataset,id,text,off_strict,off_relaxed
0,ToLD-Br,17617711318335429284,Meu nivel de amizade com isis é ela ter meu in...,True,True
1,ToLD-Br,7288264196393788121,"rt USER USER o cara adultera dados, que foram ...",True,False
2,ToLD-Br,1519599024025873488,USER USER USER o cara só é simplesmente o maio...,True,True
3,ToLD-Br,4211815168063456011,eu to chorando vei vsf e eu nem staneio izone ...,True,False
4,ToLD-Br,1231484732880705337,Eleitor do Bolsonaro é tão ignorante q não per...,True,True
...,...,...,...,...,...
27947,OLID-Br,c4eb91336485416b83d3d9de5585c74d,Essa thayse é falsa credo 😂 😂 😂 😂 😂,True,True
27948,OLID-Br,f9e864f275d14bed9cd9e2ae8aed259d,Cara que tirar de contexto USER USER falou uma...,False,False
27949,OLID-Br,574c84477a0243eba5814c4eef41aad3,USER Fala que era da era Obama!!!!! Jornalismo...,True,True
27950,OLID-Br,682e423fda2542e9a914bcf58cb44477,"Existe pessoas que tem fetiche por pessoas ""su...",False,False


In [103]:
nlp = spacy.load("pt_core_news_sm")
lemma = lambda x : x.lemma_

# Normalização dos textos
for i,row in tqdm(df.iterrows(), total=df.shape[0]):
    temp =  row['text'].lower()
    
    # Remoção de vírgulas, aspas, emojis, quebras de linha 
    temp = re.sub(r'[^\w\s]|[\n]', '',temp)
    # Lematização
    temp = " ".join(list(map(lemma, nlp(temp)))) 
    # Removeção de acentos, cedilhas e semelhantes
    temp = unidecode(temp)
    
    df.loc[i,'text'] = temp

100%|██████████| 27952/27952 [04:18<00:00, 108.23it/s]


In [339]:
OCC_TRESHOLD = 10

# Conta quantas vezes cada palavra apareceu
occurences = {}
for t in df['text']:
    words = word_tokenize(t)
    for w in words:
        if w not in occurences:
            occurences[w] = 1
        else:
            occurences[w] += 1
            
# Mantem apenas as palavras que aparecem mais vezes que o treshold
vocab = []
for w,counter in occurences.items():
    if counter > OCC_TRESHOLD:
        vocab.append(w)
        
vocab = sorted(vocab)
len(vocab)

2910